In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from IPython.display import clear_output
from numpy import arange
import os 
from sklearn.metrics import r2_score
import ta
from ta import *
import talib
#from rfpimp import permutation_importances
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import ta
import warnings
import time 
from IPython.display import clear_output
from pyti.exponential_moving_average import exponential_moving_average as EMA
import xlrd 
import xlsxwriter
import openpyxl
import sys
import decimal
import pyodbc 
import pandas.io.sql
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from ta import *
import talib
import time
import datetime
import os
#import paho.mqtt.client as mqtt
import json
# import mysql.connector
# from mysql.connector import errorcode
from elasticsearch import Elasticsearch
import socket 
import sys  
import csv
import socket
from datetime import datetime
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
# from pyspark.streaming import StreamingContext
# from pyspark.sql import SQLContext
from collections import namedtuple
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, peak_prominences
import pickle
import matplotlib
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from matplotlib import style
from functools import reduce 
import re
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn import preprocessing
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import glob


In [ ]:
#-----Loading the Original Entry Data ----#

In [2]:
path1 = "C:/Users/Administrator/WSG Markets/10 Year data creation for model/Creating model data for all strategy combined/Strategy - 1_1 TP Ratio/Original Entry Data/"
entry_data = pd.read_excel(path1+"Original Entry Data.xlsx")


In [3]:

entry_data['date'] = pd.to_datetime(entry_data['date'],format = '%Y-%m-%d %H:%M:%S')


#entry_data['Good_Bad_Flag'] = np.where(entry_data['Exit_closing_flag'] == '100_per_close',1,0)
entry_data['Good_Bad_Flag'] = np.where(entry_data['Good_Bad_Flag'] == 1,'Good','Bad')
entry_data['Year'] = entry_data['date'].dt.year
entry_data['Month'] = entry_data['date'].dt.month
entry_data['Day'] = entry_data['date'].dt.day
entry_data['Hour'] = entry_data['date'].dt.hour

entry_data = entry_data.rename(columns = {'Timeframe':'TimeFrame'})
entry_data.head()

,date,Strategy,Strategy Type,Currency,TimeFrame,Condition,_askclose,_askhigh,_askopen,_asklow,...,max_price_reached,min_price_reached,Date_updated,Exit_closing_flag,Profit_Loss_v1,SL_Flag,Year,Month,Day,Hour
0,2010-01-27 00:00:00,EMA,Normal,AUD/CHF,H1,Long,0.944400,0.94537,0.941435,0.940650,...,0.94507,na,2010-01-27 00:00:00,Stop_Loss_Reach,-100.001336,YES,2010,1,27,0
1,2010-01-28 01:00:00,EMA,Normal,AUD/CHF,H1,Long,0.943300,0.94495,0.941550,0.941450,...,0.94945,na,2010-01-28 01:00:00,100_per_close,100.000621,YES,2010,1,28,1
2,2010-02-01 19:00:00,EMA,Normal,AUD/CHF,H1,Long,0.942495,0.94257,0.941350,0.940575,...,0.9427,na,2010-02-01 19:00:00,Stop_Loss_Reach,-99.999842,YES,2010,2,1,19
3,2010-02-01 22:00:00,EMA,Normal,AUD/CHF,H1,Long,0.942400,0.94279,0.941350,0.940900,...,0.9427,na,2010-02-01 22:00:00,Stop_Loss_Reach,-99.999717,YES,2010,2,1,22
4,2010-02-09 18:00:00,EMA,Normal,AUD/CHF,H1,Long,0.934250,0.93445,0.933065,0.932100,...,1.014,na,2010-02-09 18:00:00,100_per_close,99.999982,YES,2010,2,9,18


In [ ]:
#---Loading the Indicator Data -----#

In [4]:


path2 = "C:/Users/Administrator/WSG Markets/10 Year data creation for model/Creating model data for all strategy combined/Strategy - 1_1 TP Ratio/Indicator Dataset/"



#model_data = pd.read_csv(path + "Final_data_for_model_20200630 v1.csv")
model_data = pd.read_csv(path2 + "Indicator Data.csv")
#model_data = model_data.drop(columns = ['Unnamed: 0'])
model_data['date'] = pd.to_datetime(model_data['date'])
model_data['date'] = model_data['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
model_data['date'] = pd.to_datetime(model_data['date'],format = '%Y-%m-%d %H:%M:%S')

feature_data = model_data
# feature_data['Date_updated']=pd.to_datetime(feature_data['date'],format = '%d-%m-%Y %H:%M')
feature_data['Year'] = feature_data['date'].dt.year
feature_data['Month'] = feature_data['date'].dt.month
feature_data['Day'] = feature_data['date'].dt.day
feature_data['Hour'] = feature_data['date'].dt.hour
feature_data.head()


,Currency,TimeFrame,date,askopen,askhigh,asklow,askclose,_volume,_volume_new,_stock_%K,...,MFI_overbought_sum,MFI_oversold_sum,30 Day MA,30 Day STD,Upper Band,Lower Band,Year,Month,Day,Hour
0,AUD/CHF,D1,2009-05-06 21:00:00,0.840775,0.849915,0.835195,0.846625,58440.0,58440.0,NaN,...,0,0,NaN,NaN,NaN,NaN,2009,5,6,21
1,AUD/CHF,D1,2009-05-07 21:00:00,0.846625,0.865995,0.845600,0.852275,64149.0,64149.0,NaN,...,0,0,NaN,NaN,NaN,NaN,2009,5,7,21
2,AUD/CHF,D1,2009-05-08 21:00:00,0.852275,0.857530,0.848420,0.849975,55366.0,55366.0,NaN,...,0,0,NaN,NaN,NaN,NaN,2009,5,8,21
3,AUD/CHF,D1,2009-05-11 21:00:00,0.849975,0.852390,0.839775,0.841850,54199.0,54199.0,NaN,...,0,0,NaN,NaN,NaN,NaN,2009,5,11,21
4,AUD/CHF,D1,2009-05-12 21:00:00,0.841850,0.848505,0.839700,0.845975,58175.0,58175.0,NaN,...,0,0,NaN,NaN,NaN,NaN,2009,5,12,21


In [5]:
# -------Merging Dataframe ------#

df_model_data = entry_data.merge(feature_data, on=['Currency','TimeFrame','Year','Month','Day','Hour'], how='inner')

In [6]:
df_model_data.to_csv(path1+"Entry Data with Indicators.csv",index=False)

In [7]:
df_model_data

,date_x,Strategy,Strategy Type,Currency,TimeFrame,Condition,_askclose,_askhigh,_askopen,_asklow,...,UO_overbought_sum,UO_oversold_sum,Williams %R_overbought_sum,Williams %R_oversold_sum,MFI_overbought_sum,MFI_oversold_sum,30 Day MA,30 Day STD,Upper Band,Lower Band
0,2010-01-27 00:00:00,EMA,Normal,AUD/CHF,H1,Long,0.944400,0.945370,0.941435,0.940650,...,0,0,1,0,0,0,0.939241,0.002612,0.944465,0.934017
1,2010-01-28 01:00:00,EMA,Normal,AUD/CHF,H1,Long,0.943300,0.944950,0.941550,0.941450,...,0,0,2,0,0,0,0.940222,0.001269,0.942761,0.937683
2,2010-02-01 19:00:00,EMA,Normal,AUD/CHF,H1,Long,0.942495,0.942570,0.941350,0.940575,...,0,0,5,0,0,0,0.936994,0.002833,0.942659,0.931328
3,2010-02-01 22:00:00,EMA,Normal,AUD/CHF,H1,Long,0.942400,0.942790,0.941350,0.940900,...,0,0,8,0,0,0,0.937542,0.003284,0.944110,0.930973
4,2010-02-09 18:00:00,EMA,Normal,AUD/CHF,H1,Long,0.934250,0.934450,0.933065,0.932100,...,0,0,1,0,0,0,0.931134,0.002052,0.935238,0.927029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220104,2020-11-01 22:00:00,PinscherEMA,Normal,GBP/USD,D1,Short,1.292540,1.292745,1.295095,1.291510,...,0,0,0,0,0,0,1.299515,0.006946,1.313407,1.285623
220105,2020-12-10 22:00:00,PinscherEMA,Normal,GBP/USD,D1,Short,1.329430,1.341385,1.340165,1.324545,...,0,0,0,0,0,0,1.335133,0.005523,1.346180,1.324086
220106,2020-12-22 22:00:00,PinscherEMA,Normal,GBP/USD,D1,Short,1.336070,1.347315,1.346495,1.330400,...,0,0,0,0,0,0,1.340318,0.008427,1.357171,1.323465
220107,2021-02-03 22:00:00,PinscherEMA,Normal,GBP/USD,D1,Short,1.364805,1.368350,1.366730,1.361945,...,0,0,0,0,0,0,1.366714,0.004416,1.375546,1.357883
